In [49]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.svm import SVR, NuSVR, LinearSVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import *

In [54]:
df = pd.read_csv("../data/process/train_folds.csv")
df_test = pd.read_csv("../data/original/test.csv")

df1 = pd.read_csv("../data/process/train_pred_01.csv")
df1.columns = ["id", "pred_01"]
df2 = pd.read_csv("../data/process/train_pred_02.csv")
df2.columns = ["id", "pred_02"]
df3 = pd.read_csv("../data/process/train_pred_03.csv")
df3.columns = ["id", "pred_03"]
df4 = pd.read_csv("../data/process/train_pred_04.csv")
df4.columns = ["id", "pred_04"]
df5 = pd.read_csv("../data/process/train_pred_05.csv")
df5.columns = ["id", "pred_05"]

df_test1 = pd.read_csv("../data/process/test_pred_01.csv")
df_test1.columns = ["id", "pred_01"]
df_test2 = pd.read_csv("../data/process/test_pred_02.csv")
df_test2.columns = ["id", "pred_02"]
df_test3 = pd.read_csv("../data/process/test_pred_03.csv")
df_test3.columns = ["id", "pred_03"]
df_test4 = pd.read_csv("../data/process/test_pred_04.csv")
df_test4.columns = ["id", "pred_04"]
df_test5 = pd.read_csv("../data/process/test_pred_05.csv")
df_test5.columns = ["id", "pred_05"]

df = df.merge(df1, on="id", how="left")
# df = df.merge(df2, on="id", how="left")
df = df.merge(df3, on="id", how="left")
df = df.merge(df4, on="id", how="left")
df = df.merge(df5, on="id", how="left")


df_test = df_test.merge(df_test1, on="id", how="left")
# df_test = df_test.merge(df_test2, on="id", how="left")
df_test = df_test.merge(df_test3, on="id", how="left")
df_test = df_test.merge(df_test4, on="id", how="left")
df_test = df_test.merge(df_test5, on="id", how="left")

In [55]:
sample_submission = pd.read_csv("../data/original/sample_submission.csv")
useful_features = ["pred_01", "pred_03", "pred_04","pred_05"]
df_test = df_test[useful_features]

final_test_predictions = []
final_valid_predictions = {}
scores = []
for fold in range(5):
    xtrain =  df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)
    xtest = df_test.copy()

    valid_ids = xvalid.id.values.tolist()

    ytrain = xtrain.target
    yvalid = xvalid.target
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    

    params = {
        'random_state': 1, 
        'booster': 'gbtree',
        'n_estimators': 7000,
        'learning_rate': 0.03,
        'max_depth': 2
    }
    
    model = XGBRegressor(
        n_jobs=4,
        **params
    )
    model.fit(xtrain, ytrain, early_stopping_rounds=300, eval_set=[(xvalid, yvalid)], verbose=1000)
    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)
    final_test_predictions.append(test_preds)
    final_valid_predictions.update(dict(zip(valid_ids, preds_valid)))
    rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    print(fold, rmse)
    scores.append(rmse)

print(np.mean(scores), np.std(scores))
final_valid_predictions = pd.DataFrame.from_dict(final_valid_predictions, orient="index").reset_index()
final_valid_predictions.columns = ["id", "pred_1"]
final_valid_predictions.to_csv("level1_train_pred_1.csv", index=False)

sample_submission.target = np.mean(np.column_stack(final_test_predictions), axis=1)
sample_submission.columns = ["id", "pred_1"]
sample_submission.to_csv("level1_test_pred_1.csv", index=False)

[0]	validation_0-rmse:7.54855
[588]	validation_0-rmse:0.71550
0 0.7154563868940557
[0]	validation_0-rmse:7.54526
[551]	validation_0-rmse:0.71554
1 0.7155342412269988
[0]	validation_0-rmse:7.54304
[537]	validation_0-rmse:0.71750
2 0.7174474488043614
[0]	validation_0-rmse:7.54532
[556]	validation_0-rmse:0.71742
3 0.7174062498700212
[0]	validation_0-rmse:7.55076
[611]	validation_0-rmse:0.71542
4 0.7153733166290285
0.716243528684893 0.0009676046845057461


In [56]:
sample_submission = pd.read_csv("../data/original/sample_submission.csv")
useful_features = ["pred_01", "pred_03", "pred_04", "pred_05"]
df_test = df_test[useful_features]

final_test_predictions = []
final_valid_predictions = {}
scores = []
for fold in range(5):
    xtrain =  df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)
    xtest = df_test.copy()

    valid_ids = xvalid.id.values.tolist()

    ytrain = xtrain.target
    yvalid = xvalid.target
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    
    model = RandomForestRegressor(n_estimators=500, n_jobs=-1, max_depth=3)
    model.fit(xtrain, ytrain)
    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)
    final_test_predictions.append(test_preds)
    final_valid_predictions.update(dict(zip(valid_ids, preds_valid)))
    rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    print(fold, rmse)
    scores.append(rmse)

print(np.mean(scores), np.std(scores))
final_valid_predictions = pd.DataFrame.from_dict(final_valid_predictions, orient="index").reset_index()
final_valid_predictions.columns = ["id", "pred_2"]
final_valid_predictions.to_csv("level1_train_pred_2.csv", index=False)

sample_submission.target = np.mean(np.column_stack(final_test_predictions), axis=1)
sample_submission.columns = ["id", "pred_2"]
sample_submission.to_csv("level1_test_pred_2.csv", index=False)

0 0.7157847010339962
1 0.7158477403486432
2 0.7177142089955655
3 0.7178249318344556
4 0.7157167982707114
0.7165776760966744 0.0009746876610697496


In [57]:
sample_submission = pd.read_csv("../data/original/sample_submission.csv")
useful_features = ["pred_01", "pred_03", "pred_04", "pred_05"]
df_test = df_test[useful_features]

final_test_predictions = []
final_valid_predictions = {}
scores = []
for fold in range(5):
    xtrain =  df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)
    xtest = df_test.copy()

    valid_ids = xvalid.id.values.tolist()

    ytrain = xtrain.target
    yvalid = xvalid.target
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    
    cat_parameters= {    
    'iterations':1600,
    'learning_rate':0.024,
    'l2_leaf_reg':20,
    'random_strength':1.5,
    'grow_policy':'Depthwise',
    'leaf_estimation_method':'Newton', 
    'bootstrap_type':'Bernoulli',
    'thread_count':4,
    'verbose':False,
    'loss_function':'RMSE',
    'eval_metric':'RMSE',
    'od_type':'Iter'
}
    
    model = CatBoostRegressor(**cat_parameters)
    model.fit(xtrain, ytrain)
    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)
    final_test_predictions.append(test_preds)
    final_valid_predictions.update(dict(zip(valid_ids, preds_valid)))
    rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    print(fold, rmse)
    scores.append(rmse)

print(np.mean(scores), np.std(scores))
final_valid_predictions = pd.DataFrame.from_dict(final_valid_predictions, orient="index").reset_index()
final_valid_predictions.columns = ["id", "pred_5"]
final_valid_predictions.to_csv("level1_train_pred_5.csv", index=False)

sample_submission.target = np.mean(np.column_stack(final_test_predictions), axis=1)
sample_submission.columns = ["id", "pred_5"]
sample_submission.to_csv("level1_test_pred_5.csv", index=False)

0 0.7161333657716928
1 0.7162985074142836
2 0.7180024187221201
3 0.7180284754016436
4 0.7162937058245143
0.7169512946268509 0.0008709468497202969


In [58]:
sample_submission = pd.read_csv("../data/original/sample_submission.csv")
useful_features = ["pred_01", "pred_03", "pred_04", "pred_05"]
df_test = df_test[useful_features]

final_test_predictions = []
final_valid_predictions = {}
scores = []
for fold in range(5):
    xtrain =  df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)
    xtest = df_test.copy()

    valid_ids = xvalid.id.values.tolist()

    ytrain = xtrain.target
    yvalid = xvalid.target
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    
    model = GradientBoostingRegressor(n_estimators=500, max_depth=3)
    model.fit(xtrain, ytrain)
    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)
    final_test_predictions.append(test_preds)
    final_valid_predictions.update(dict(zip(valid_ids, preds_valid)))
    rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    print(fold, rmse)
    scores.append(rmse)

print(np.mean(scores), np.std(scores))
final_valid_predictions = pd.DataFrame.from_dict(final_valid_predictions, orient="index").reset_index()
final_valid_predictions.columns = ["id", "pred_3"]
final_valid_predictions.to_csv("level1_train_pred_3.csv", index=False)

sample_submission.target = np.mean(np.column_stack(final_test_predictions), axis=1)
sample_submission.columns = ["id", "pred_3"]
sample_submission.to_csv("level1_test_pred_3.csv", index=False)

0 0.7160648789242821
1 0.7160899440360843
2 0.7180359681658036
3 0.7180717553243343
4 0.7160982252921594
0.7168721543485327 0.0009649888984038619


In [60]:
sample_submission = pd.read_csv("../data/original/sample_submission.csv")
useful_features = ["pred_01", "pred_03", "pred_04", "pred_05"]
df_test = df_test[useful_features]

final_test_predictions = []
final_valid_predictions = {}
scores = []
for fold in range(5):
    xtrain =  df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)
    xtest = df_test.copy()

    valid_ids = xvalid.id.values.tolist()

    ytrain = xtrain.target
    yvalid = xvalid.target
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    
    model = LGBMRegressor(n_estimators = 10000, learning_rate = 0.1, random_state = fold, max_depth = 2,
                          subsample = 0.95, colsample_bytree = 0.85, reg_alpha = 30.0, reg_lambda = 25.0
                          , num_leaves = 4, max_bin = 512)
    model.fit(xtrain, ytrain)
    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)
    final_test_predictions.append(test_preds)
    final_valid_predictions.update(dict(zip(valid_ids, preds_valid)))
    rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    print(fold, rmse)
    scores.append(rmse)

print(np.mean(scores), np.std(scores))
final_valid_predictions = pd.DataFrame.from_dict(final_valid_predictions, orient="index").reset_index()
final_valid_predictions.columns = ["id", "pred_4"]
final_valid_predictions.to_csv("level1_train_pred_4.csv", index=False)

sample_submission.target = np.mean(np.column_stack(final_test_predictions), axis=1)
sample_submission.columns = ["id", "pred_4"]
sample_submission.to_csv("level1_test_pred_4.csv", index=False)

0 0.7157044910124101
1 0.7159569603458641
2 0.7177279013579634
3 0.7177502139526108
4 0.7157308150645632
0.7165740763466824 0.0009552708753757295


In [116]:
df = pd.read_csv("../data/process/train_folds.csv")
df_test = pd.read_csv("../data/original/test.csv")
sample_submission = pd.read_csv("../data/original/sample_submission.csv")

df1 = pd.read_csv("level1_train_pred_1.csv")
df2 = pd.read_csv("level1_train_pred_2.csv")
df3 = pd.read_csv("level1_train_pred_3.csv")
df4 = pd.read_csv("level1_train_pred_4.csv")
df5 = pd.read_csv("level1_train_pred_5.csv")

df_test1 = pd.read_csv("level1_test_pred_1.csv")
df_test2 = pd.read_csv("level1_test_pred_2.csv")
df_test3 = pd.read_csv("level1_test_pred_3.csv")
df_test4 = pd.read_csv("level1_test_pred_4.csv")
df_test5 = pd.read_csv("level1_test_pred_5.csv")

df = df.merge(df1, on="id", how="left")
df = df.merge(df2, on="id", how="left")
df = df.merge(df3, on="id", how="left")
df = df.merge(df4, on="id", how="left")
df = df.merge(df5, on="id", how="left")

df_test = df_test.merge(df_test1, on="id", how="left")
df_test = df_test.merge(df_test2, on="id", how="left")
df_test = df_test.merge(df_test3, on="id", how="left")
df_test = df_test.merge(df_test4, on="id", how="left")
df_test = df_test.merge(df_test5, on="id", how="left")

df.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont11,cont12,cont13,target,kfold,pred_1,pred_2,pred_3,pred_4,pred_5
0,1,B,B,B,C,B,B,A,E,C,...,0.377873,0.322401,0.869850,8.113634,0,8.467678,8.458017,8.462344,8.443071,8.437379
1,2,B,B,A,A,B,D,A,F,A,...,0.921701,0.261975,0.465083,8.481233,2,8.472032,8.468426,8.476156,8.462179,8.455944
2,3,A,A,A,C,B,D,A,D,A,...,0.620126,0.541474,0.763846,8.364351,4,8.227381,8.237577,8.234507,8.222404,8.227418
3,4,B,B,A,C,B,D,A,E,C,...,0.714610,0.540150,0.280682,8.049253,3,8.353996,8.343003,8.360189,8.367214,8.368331
4,6,A,A,A,C,B,D,A,E,A,...,0.776742,0.625849,0.250823,7.972260,1,8.275207,8.255831,8.283782,8.289361,8.291144


In [117]:
useful_features = ["pred_1","pred_2","pred_3"]
df_test = df_test[useful_features]

final_predictions = []
scores = []
for fold in range(5):
    xtrain =  df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)
    xtest = df_test.copy()

    ytrain = xtrain.target
    yvalid = xvalid.target
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    
    model = LinearRegression()
    model.fit(xtrain, ytrain)
    
    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)
    final_predictions.append(test_preds)
    rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    print(fold, rmse)
    scores.append(rmse)

print(np.mean(scores), np.std(scores))

0 0.715448762169117
1 0.7155374773859909
2 0.7174603508912751
3 0.7174253316987551
4 0.7153395267466436
0.7162422897783565 0.0009823124479217298


0 0.7154462094886919
1 0.7155485216959859
2 0.7174406321103276
3 0.7174416816322292
4 0.7153393674615113
0.7162432824777492 0.0009802945400679895

In [98]:
useful_features = ["pred_1"]
df_test = df_test[useful_features]

final_predictions = []
scores = []
for fold in range(5):
    xtrain =  df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)
    xtest = df_test.copy()

    ytrain = xtrain.target
    yvalid = xvalid.target
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    
    model = Ridge(0.5)
    model.fit(xtrain, ytrain)
    
    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)
    final_predictions.append(test_preds)
    rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    print(fold, rmse)
    scores.append(rmse)

print(np.mean(scores), np.std(scores))

0 0.7154471902068562
1 0.7155336300506926
2 0.7174584403374683
3 0.7174228344929512
4 0.7153538272697599
0.7162431844715457 0.0009794336700971876


0 0.7154462094745094
1 0.7155485216582944
2 0.7174406320290153
3 0.7174416816596121
4 0.7153393675562559
0.7162432824755374 0.0009802945170761818

In [118]:
sample_submission.target = np.mean(np.column_stack(final_predictions), axis=1)
sample_submission.to_csv("../data/submit/024_submission.csv", index=False)

In [29]:
useful_features = ["pred_1", "pred_2", "pred_3", "pred_4", "pred_5"]
df_test = df_test[useful_features]

final_predictions = []
scores = []
for fold in range(5):
    xtrain =  df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)
    xtest = df_test.copy()

    ytrain = xtrain.target
    yvalid = xvalid.target
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    
    model = Ridge(alpha = 0.25)
    model.fit(xtrain, ytrain)
    
    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)
    final_predictions.append(test_preds)
    rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    print(fold, rmse)
    scores.append(rmse)

print(np.mean(scores), np.std(scores))

0 0.7154461786196353
1 0.7155484328730373
2 0.7174404349933494
3 0.717441753555
4 0.71533960802995
0.7162432816141944 0.000980237227410213


In [30]:
sample_submission.target = np.mean(np.column_stack(final_predictions), axis=1)
sample_submission.to_csv("../data/submit/019_submission.csv", index=False)

In [43]:
preds = [df_test1, df_test2, df_test3, df_test4]
weights = [0.25,0.25,0.25,0.25] 

sample_submission = pd.read_csv("../data/original/sample_submission.csv")
sample_submission.target = 0.0

for pred, weight in zip(preds, weights):
    sample_submission.target += weight * pred / sum(weights)

sample_submission.to_csv('../data/submit/020_submission.csv', index=False)

AttributeError: 'NotImplementedType' object has no attribute '_indexed_same'

In [88]:
pred_ave = df_test.apply(gmean,axis=1)

In [78]:
pred_ave = df_test.sum(axis=1)/5 

In [79]:
pred_ave.head()

0    8.049925
1    8.448029
2    8.433083
3    8.489623
4    8.137091
dtype: float64

In [91]:
# sample_submission.target = np.mean(np.column_stack(final_predictions), axis=1)
sample_submission.to_csv("../data/submit/022_submission.csv", index=False)

In [89]:
sample_submission["target"] = pred_ave

In [90]:
sample_submission

,id,target
0,0,8.049914
1,5,8.448004
2,15,8.433077
3,16,8.489508
4,17,8.137081
...,...,...
199995,499987,7.956875
199996,499990,8.484645
199997,499991,8.588285
199998,499994,8.173422


In [84]:
from scipy.stats.mstats import gmean